<a href="https://colab.research.google.com/github/flavioluizseixas/ExtractiveQA/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!sudo apt install build-essential libpoppler-cpp-dev pkg-config python3-dev
!sudo apt install xpdf
!pip install farm-haystack[colab,inference]
!pip install haystack
!pip install langdetect
!pip install nltk
!pip install pdftotext

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
build-essential is already the newest version (12.9ubuntu3).
pkg-config is already the newest version (0.29.2-1ubuntu3).
libpoppler-cpp-dev is already the newest version (22.02.0-2ubuntu0.3).
python3-dev is already the newest version (3.10.6-1~22.04).
0 upgraded, 0 newly installed, 0 to remove and 33 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
xpdf is already the newest version (3.04+git20220201-1).
0 upgraded, 0 newly installed, 0 to remove and 33 not upgraded.


In [2]:
from haystack.nodes import PDFToTextConverter

pdf_converter = PDFToTextConverter(remove_numeric_tables=True, valid_languages=["en"])

converted = pdf_converter.convert(file_path = "annual-report-adidas-ar20.pdf",
                                  meta = { "company": "Company_1", "processed": False })

In [3]:
from haystack.nodes import PreProcessor

preprocessor = PreProcessor(split_by="word",
                            split_length=200,
                            split_overlap=10)

preprocessed = preprocessor.process(converted)

Preprocessing: 100%|██████████| 1/1 [00:00<00:00,  3.12docs/s]


In [4]:
#from haystack.document_stores import InMemoryDocumentStore
#document_store = InMemoryDocumentStore()
#document_store.write_documents(preprocessed)

from haystack.document_stores import InMemoryDocumentStore
document_store = InMemoryDocumentStore(use_bm25=True)
document_store.write_documents(preprocessed)

In [5]:
#from haystack.nodes import DensePassageRetriever
#retriever = DensePassageRetriever(document_store=document_store)

from haystack.nodes import BM25Retriever
retriever = BM25Retriever(document_store=document_store)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/493 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/492 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [6]:
from haystack.nodes import FARMReader

reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)

In [7]:
from haystack.pipelines import ExtractiveQAPipeline

pipe = ExtractiveQAPipeline(reader, retriever)

In [8]:
#document_store.update_embeddings(retriever)

Create embeddings: 100%|██████████| 832/832 [08:42<00:00,  1.51 Docs/s]
Documents Processed: 10000 docs [08:43, 19.11 docs/s]


In [9]:
questions = [ "How high is shareholders equity?",
              "What are the major risks?",
              "What is the number of shares outstanding?",
              "How high is short term debt?" ]

In [10]:
answers = []

for question in questions:
    prediction = pipe.run(query=question,
                 params = {"Retriever": {"top_k": 50},
                           "Reader": { "top_k": 1 } })
    answers.append(prediction)

Inferencing Samples: 100%|██████████| 2/2 [00:50<00:00, 25.26s/ Batches]


In [11]:
for answer in answers:
    print("Q:", answer['query'])
    print("A:", answer['answers'][0].answer)
    print("\n")

Q: How high is shareholders equity?
A: 7
027


Q: What are the major risks?
A: accounts receivable and, to a lesser extent, from
other third-party contractual financial obligations such as other financial assets, short-term bank deposits
and derivative financial instruments


Q: What is the number of shares outstanding?
A: 195,066,060


Q: How high is short term debt?
A: € 686 million


